# Set up Model Predictive Control Using Do-MPC Package

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import casadi as cas
import do_mpc

from cas_models.continuous_time.models import StateSpaceModelCT
from cas_models.transformations import connect_systems
from feed_conc_ctrl.models import MixingTankModelCT, RatioControlledFlowMixerCT

## Example: Two-Tank Feed Blending System

```none
        ┌────────┐        
  ─────►┤ Tank 1 │        
        │        │        
        │        ├──┐     
        └────────┘  │     
                    │     
                    │     
                    ├────►
        ┌────────┐  │     
  ─────►┤ Tank 2 │  │     
        │        │  │     
        │        ├──┘     
        └────────┘        
```

States
 1. Tank 1 level
 2. Tank 1 mass
 3. Tank 2 level
 4. Tank 2 mass

Inputs:
 1. Tank 1 inflow rate
 2. Tank 1 inflow concentration
 3. Tank 1 outflow rate
 4. Tank 2 inflow rate
 5. Tank 2 inflow concentration
 6. Tank 2 outflow rate

Outputs:
 1. Tank 1 level
 2. Tank 1 mass
 3. Tank 1 outflow concentration
 4. Tank 2 level
 5. Tank 2 mass
 6. Tank 2 outflow concentration
 7. Mixer outflow rate
 8. Mixer outflow concentration

In [2]:
def print_sys_dimensions(sys):
    print(sys.name, f"({sys.ny}x{sys.nu})")
    for attr_name in ["input_names", "state_names", "output_names"]:
        print(f"{attr_name:>15s}: {getattr(sys, attr_name)}")

## Construct Mixing Tank System Model using Casadi-Models Package

In [3]:
# Test new ratio controller
mixer = RatioControlledFlowMixerCT(2)
assert (
    str(mixer) == (
        "RatioControlledFlowMixerCT("
        "f=Function(f:(t,x[0],u[4])->(rhs[0]) SXFunction), "
        "h=Function(h:(t,x[0],u[4])->(y[3]) SXFunction), "
        "n=0, nu=4, ny=3, params={}, name='FlowMixerModel', "
        "input_names=['r_1', 'conc_in_1', 'v_dot_out', 'conc_in_2'], "
        "state_names=[], "
        "output_names=['v_dot_in_1', 'v_dot_in_2', 'conc_out'])")
)

t = 0.0
x = []

# Equal inflow concentrations, 50:50 ratio
u = [0.5, 10.0, 2.0, 10.0]
y = mixer.h(t, x, u)
assert np.allclose(np.array(y).flatten(), [1, 1, 10])

# Equal inflow concentrations, 20:80 ratio
u = [0.2, 10.0, 2.0, 10.0]
y = mixer.h(t, x, u)
assert np.allclose(np.array(y).flatten(), [0.4, 1.6, 10])

# Different concentrations, 50:50 ratio
u = [0.5, 10.0, 2.0, 0.0]
y = mixer.h(t, x, u)
assert np.allclose(np.array(y).flatten(), [1, 1, 5])

#  Mixer wit 3 inputs
mixer = RatioControlledFlowMixerCT(3)
assert (
    str(mixer) == (
        "RatioControlledFlowMixerCT("
        "f=Function(f:(t,x[0],u[6])->(rhs[0]) SXFunction), "
        "h=Function(h:(t,x[0],u[6])->(y[4]) SXFunction), "
        "n=0, nu=6, ny=4, params={}, name='FlowMixerModel', "
        "input_names=['r_1', 'conc_in_1', 'r_2', 'conc_in_2', 'v_dot_out', 'conc_in_3'], "
        "state_names=[], "
        "output_names=['v_dot_in_1', 'v_dot_in_2', 'v_dot_in_3', 'conc_out'])")
)

u = [0.3, 10.0, 0.4, 20.0, 2.0, 30.0]
y = mixer.h(t, x, u)
assert np.allclose(
    np.array(y).flatten(), 
    [0.3 * 2, 0.4 * 2, 0.3 * 2, (0.3 * 10 + 0.4 * 20 + 0.3 * 30)]
)

In [ ]:
D = 5  # tank diameter [m]
n_tanks = 2
tank_names = [f"tank_{i+1}" for i in range(n_tanks)]

# Initialize tank system models
systems = [MixingTankModelCT(D=D, name=name) for name in tank_names]

# Add a flow mixer to join flows from two tanks
systems.append(RatioControlledFlowMixerCT(2, name="mixer"))

# Check inputs and outputs
for sys in systems:
    print_sys_dimensions(sys)

In [ ]:
# Connect all systems together
connections = {
    'mixer_conc_in_1': 'tank_1_conc_out',
    'mixer_conc_in_2': 'tank_2_conc_out',
    'tank_1_v_dot_out': 'mixer_v_dot_in_1',
    'tank_2_v_dot_out': 'mixer_v_dot_in_2',
}

model_class = StateSpaceModelCT
feed_tanks_system = connect_systems(
    systems, connections, model_class, name="tank_system_21", verbose_names=True
) 
print_sys_dimensions(feed_tanks_system)

## Control Model Design

**States**
 1. Tank 1 level
 2. Tank 1 mass
 3. Tank 2 level
 4. Tank 2 mass

**Manipulated Variables (MVs)**
 1. Tank 1 inflow rate
 2. Tank 2 inflow rate
 3. Mixer inflow ratio

where mixer inflow ratio = tank 1 outflow rate / 
    (tank 1 outflow rate + tank 2 outflow rate)

**Unmeasured Disturbances**
 1. Tank 1 inflow concentration
 2. Tank 2 inflow concentration
 3. Mixer outflow rate

**Controlled Variables (CVs)**
 1. Tank 1 level
 2. Tank 1 outflow concentration
 3. Tank 2 level
 4. Tank 2 outflow concentration
 5. Mixer outflow concentration



## Construct Do-MPC Model from Casadi System Model

In [6]:
system = feed_tanks_system
control_design = {
    "manipulated_variables": ['tank_1_v_dot_in', 'tank_2_v_dot_in', 'mixer_r_1'],
    "unmeasured_disturbances": ['tank_1_conc_in', 'tank_2_conc_in', 'mixer_v_dot_out'],
    "controlled_variables": [
        'tank_1_L', 'tank_1_conc_out', 'tank_2_L', 'tank_2_conc_out', 'mixer_conc_out'
    ],
}

# Check all inputs accounted for
assert set(system.input_names) == set(
    control_design["manipulated_variables"] + control_design["unmeasured_disturbances"]
)

model_type = 'continuous'
model = do_mpc.model.Model(model_type)

# Add manipulated variables (MVs)
for name in control_design["manipulated_variables"]:
    model.set_variable(var_type='_u', var_name=name, shape=(1, 1))

# Add all state variables in same order as system
for name in system.state_names:
    model.set_variable(var_type='_x', var_name=name, shape=(1, 1))

# Augment model with additional states for unmeasured disturbances
for name in control_design["unmeasured_disturbances"]:
    model.set_variable(var_type='_x', var_name=name, shape=(1, 1))

# TODO: How to add measured disturbances?

t = 0  # assume time invariant
states = cas.vcat(model.x[system.state_names])

inputs = []
for name in system.input_names:
    if name in control_design['manipulated_variables']:
        inputs.append(model.u[name])
    elif name in control_design.get('measured_disturbances', []):
        inputs.append(model.x[name])
    elif name in control_design.get('unmeasured_disturbances', []):
        inputs.append(model.x[name])
inputs = cas.vcat(inputs)
# TODO: Deal with params

rhs = system.f(t, states, inputs)
outputs = system.h(t, states, inputs)

# Set righthand-side expressions for system states
for i, name in enumerate(system.state_names):
    model.set_rhs(name, rhs[i])

# Set righthand-side expressions for unmeasured disturbances
for i, name in enumerate(control_design['unmeasured_disturbances']):
    model.set_rhs(
        name,
        cas.DM(0)  # d_dot = 0 + process_noise (added by MHE)
    )
    # TODO: Allow alternative disturbance models

# Define measured variables and output expressions
for name in control_design["controlled_variables"]:
    i = system.output_names.index(name)
    model.set_meas(meas_name=name, expr=outputs[i])